In [4]:
import rope.base.project
from rope.refactor.extract import ExtractMethod
from rope.refactor.inline import InlineMethod
import re

APOSTROPHE_MARKER = "__APOSTROPHE__"
PERCENT_FORMAT_MARKER = "__PERCENT_FORMAT__"

def serialize(text: str):
    # Replace "'{var}'" with "__APOSTROPHE__{var}__APOSTROPHE__"
    text = re.sub(r"'{([^'}]*?)}'", f"{APOSTROPHE_MARKER}{{\\1}}{APOSTROPHE_MARKER}", text)
    # Replace "%s" with "__PERCENT_FORMAT__"
    text = re.sub(r"%\((.*?)\)s", f"{PERCENT_FORMAT_MARKER}{{\\1}}", text)
    return text

def deserialize(text: str):
    text = re.sub(f"{APOSTROPHE_MARKER}{{(.*?)}}{APOSTROPHE_MARKER}", "'{\\1}'", text)
    text = re.sub(f"{PERCENT_FORMAT_MARKER}{{(.*?)}}", "%(\\1)s", text)
    return text

myproject = rope.base.project.Project('src')

myresource = myproject.get_resource('test.py')
contents = myresource.read()
print(contents)
serialized_contents = serialize(myresource.read())
myresource.write(serialized_contents)
extract_span = r"""new_function_names = []
        existing_names = ", ".join(
            [def_fn.name.strip("'") for def_fn in all_defined_functions]
        )
        offset = 5
        for idx in range(0, len(deduped_exact_matches), offset):
            num_snippets = min(len(deduped_exact_matches), idx + offset) - idx
            formatted_snippets = "\n".join(
                [
                    f"<function_to_name>\n{snippet}\n</function_to_name>"
                    for snippet in deduped_exact_matches[idx:idx+num_snippets]
                ]
            )
            new_function_names.extend(NameBot(chat_logger=self.chat_logger).name_functions(
                old_code=cloned_repo.get_file_contents(file_path),
                snippets=formatted_snippets,
                existing_names=existing_names,
                count=num_snippets,
            ))
        for idx, extracted_original_code in enumerate(deduped_exact_matches):
            if idx >= len(new_function_names):
                break"""
serialized_extract_span = serialize(extract_span)
print(serialized_extract_span)

start, end = serialized_contents.find(serialized_extract_span), serialized_contents.find(serialized_extract_span) + len(serialized_extract_span)
print(start, end)

try:
    extractor = ExtractMethod(myproject, myresource, start, end)
    change_set = extractor.get_changes("helper", similar=True)
    for change in change_set.changes:
        if change.old_contents is not None:
            change.old_contents = deserialize(change.old_contents)
        else:
            change.old_contents = deserialize(change.resource.read())
        change.new_contents = deserialize(change.new_contents)
    for change in change_set.changes:
        print(change.get_description())
except Exception as e:
    print(e)
    raise e
finally:
    myresource.write(contents)

import re
from regex import D

import rope.base.project
from loguru import logger
from rope.refactor.extract import ExtractMethod

from sweepai.agents.name_agent import NameBot
from sweepai.config.server import DEFAULT_GPT4_32K_MODEL, DEFAULT_GPT35_MODEL
from sweepai.core.chat import ChatGPT
from sweepai.core.entities import Message
from sweepai.core.update_prompts import (
    extract_snippets_system_prompt,
    extract_snippets_user_prompt,
)
from sweepai.utils.github_utils import ClonedRepo
from sweepai.utils.jedi_utils import (
    get_all_defined_functions,
    get_references_from_defined_function,
    setup_jedi_for_file,
)
from sweepai.utils.refactor_utils import get_refactor_snippets
from sweepai.utils.search_and_replace import find_best_match

APOSTROPHE_MARKER = "__APOSTROPHE__"
PERCENT_FORMAT_MARKER = "__PERCENT_FORMAT__"


def serialize(text: str):
    # Replace "'{var}'" with "__APOSTROPHE__{var}__APOSTROPHE__"
    text = re.sub(
        r"'{([^'}]*?)}'", f"{APOSTROPHE_MAR

In [ ]:
myresource = myproject.get_resource('mod3.py')
contents = myresource.read()
serialized_contents = serialize(myresource.read())
myresource.write(serialized_contents)
extract_span = r"""                openai.api_type = OPENAI_API_TYPE
                openai.api_base = OPENAI_API_BASE
                openai.api_version = OPENAI_API_VERSION
                openai.api_key = AZURE_API_KEY
                response = openai.ChatCompletion.create(
                    engine=engine,
                    model=model,
                    messages=messages,
                    max_tokens=max_tokens,
                    temperature=temperature,
                    timeout=OPENAI_TIMEOUT,
                )"""
serialized_extract_span = serialize(extract_span)
print(serialized_extract_span)

start, end = serialized_contents.find(serialized_extract_span), serialized_contents.find(serialized_extract_span) + len(serialized_extract_span)
print(start, end)

try:
    extractor = ExtractMethod(myproject, myresource, start, end)
    change_set = extractor.get_changes("helper", similar=True)
    for change in change_set.changes:
        if change.old_contents is not None:
            change.old_contents = deserialize(change.old_contents)
        else:
            change.old_contents = deserialize(change.resource.read())
        change.new_contents = deserialize(change.new_contents)
    for change in change_set.changes:
        print(change.get_description())
except Exception as e:
    print(e)
finally:
    myresource.write(contents)

In [22]:
len(change.new_contents) - len(change.old_contents)

85

In [15]:
myproject = rope.base.project.Project('./src')
mod1 = myproject.get_resource('mod3.py')
mod1.write(r"""
class OpenAIProxy:
    def __init__(self):
        pass

    @file_cache(ignore_params=[])
    def call_openai(self, model, messages, max_tokens, temperature) -> str:
        try:
            engine = None
            if model in OPENAI_EXCLUSIVE_MODELS and OPENAI_API_TYPE != "azure":
                logger.info(f"Calling OpenAI exclusive model. {model}")
                raise Exception("OpenAI exclusive model.")
            if (
                model == "gpt-3.5-turbo-16k"
                or model == "gpt-3.5-turbo-16k-0613"
                and OPENAI_API_ENGINE_GPT35 is not None
            ):
                engine = OPENAI_API_ENGINE_GPT35
            elif (
                model == "gpt-4"
                or model == "gpt-4-0613"
                and OPENAI_API_ENGINE_GPT4 is not None
            ):
                engine = OPENAI_API_ENGINE_GPT4
            elif (
                model == "gpt-4-32k"
                or model == "gpt-4-32k-0613"
                and OPENAI_API_ENGINE_GPT4_32K is not None
            ):
                engine = OPENAI_API_ENGINE_GPT4_32K
            if OPENAI_API_TYPE is None or engine is None:
                openai.api_key = OPENAI_API_KEY
                openai.api_base = "https://api.openai.com/v1"
                openai.api_version = None
                openai.api_type = "open_ai"
                response = openai.ChatCompletion.create(
                    model=model,
                    messages=messages,
                    max_tokens=max_tokens,
                    temperature=temperature,
                    timeout=OPENAI_TIMEOUT,
                    seed=SEED,
                )
                return response["choices"][0].message.content
            # validity checks for MULTI_REGION_CONFIG
            if (
                MULTI_REGION_CONFIG is None
                or not isinstance(MULTI_REGION_CONFIG, list)
                or len(MULTI_REGION_CONFIG) == 0
                or not isinstance(MULTI_REGION_CONFIG[0], list)
            ):
                logger.info(
                    f"Calling {model} with engine {engine} on Azure url {OPENAI_API_BASE}."
                )
                openai.api_type = OPENAI_API_TYPE
                openai.api_base = OPENAI_API_BASE
                openai.api_version = OPENAI_API_VERSION
                openai.api_key = AZURE_API_KEY
                response = openai.ChatCompletion.create(
                    engine=engine,
                    model=model,
                    messages=messages,
                    max_tokens=max_tokens,
                    temperature=temperature,
                    timeout=OPENAI_TIMEOUT,
                )
                return response["choices"][0].message.content
            # multi region config is a list of tuples of (region_url, api_key)
            # we will try each region in order until we get a response
            # randomize the order of the list
            SHUFFLED_MULTI_REGION_CONFIG = random.sample(
                MULTI_REGION_CONFIG, len(MULTI_REGION_CONFIG)
            )
            for region_url, api_key in SHUFFLED_MULTI_REGION_CONFIG:
                try:
                    logger.info(
                        f"Calling {model} with engine {engine} on Azure url {region_url}."
                    )
                    openai.api_key = api_key
                    openai.api_base = region_url
                    openai.api_version = OPENAI_API_VERSION
                    openai.api_type = OPENAI_API_TYPE
                    response = openai.ChatCompletion.create(
                        engine=engine,
                        model=model,
                        messages=messages,
                        max_tokens=max_tokens,
                        temperature=temperature,
                        timeout=OPENAI_TIMEOUT,
                    )
                    return response["choices"][0].message.content
                except SystemExit:
                    raise SystemExit
                except Exception as e:
                    logger.exception(f"Error calling {region_url}: {e}")
            raise Exception("No Azure regions available")
        except SystemExit:
            raise SystemExit
        except Exception as e:
            if OPENAI_API_KEY:
                try:
                    openai.api_key = OPENAI_API_KEY
                    openai.api_base = "https://api.openai.com/v1"
                    openai.api_version = None
                    openai.api_type = "open_ai"
                    response = openai.ChatCompletion.create(
                        model=model,
                        messages=messages,
                        max_tokens=max_tokens,
                        temperature=temperature,
                        timeout=OPENAI_TIMEOUT,
                        seed=SEED,
                    )
                    return response["choices"][0].message.content
                except SystemExit:
                    raise SystemExit
                except Exception as _e:
                    logger.error(f"OpenAI API Key found but error: {_e}")
            logger.error(f"OpenAI API Key not found and Azure Error: {e}")
            # Raise exception to report error
            raise e
""")

from rope.refactor import restructure
pattern = r"""
openai.api_key = ${api_key}
openai.api_base = ${api_base}
openai.api_version = ${api_version}
openai.api_type = ${api_type}
response = openai.ChatCompletion.create(
    model=${model},
    messages=${messages},
    max_tokens=${max_tokens},
    temperature=${temperature},
    timeout=${timeout},
    seed=${seed},
)"""
goal = """print(
    ${api_key},
    ${api_base},
    ${api_version},
    ${api_type},
    ${model},
    ${messages},
    ${max_tokens},
    ${temperature},
    ${timeout},
    ${seed},
)"""

restructuring = restructure.Restructure(myproject, pattern, goal)
print(restructuring.get_changes().get_description())

Restructuring <
openai.api_key = ${api_key}
openai.api_base = ${api_base}
openai.api_version = ${api_version}
openai.api_type = ${api_type}
logger.info(f"Calling {model} with OpenAI.")
response = openai.ChatCompletion.create(
    model=${model},
    messages=${messages},
    max_tokens=${max_tokens},
    temperature=${temperature},
    timeout=${timeout},
    seed=${seed},
)> to <print(
    ${api_key},
    ${api_base},
    ${api_version},
    ${api_type},
    ${model},
    ${messages},
    ${max_tokens},
    ${temperature},
    ${timeout},
    ${seed},
)>:


--- a/mod3.py
+++ b/mod3.py
@@ -29,18 +29,17 @@
             ):
                 engine = OPENAI_API_ENGINE_GPT4_32K
             if OPENAI_API_TYPE is None or engine is None:
-                openai.api_key = OPENAI_API_KEY
-                openai.api_base = "https://api.openai.com/v1"
-                openai.api_version = None
-                openai.api_type = "open_ai"
-                logger.info(f"Calling {model} with OpenAI.

In [57]:
from rope.refactor import usefunction

sweep_project = rope.base.project.Project('../../sweepai', ignored_resources=["sandbox"])
resource = sweep_project.get_resource('utils/str_utils.py')
# resource = sweep_project.get_resource('utils/openai_proxy.py')
# start = resource.read().find('_call_openai')
start = resource.read().find('entities_split')
use_function = usefunction.UseFunction(sweep_project, resource, start)
print(use_function.get_changes().get_description())

Using function <_call_openai>:


--- a/utils/openai_proxy.py
+++ b/utils/openai_proxy.py
@@ -60,18 +60,7 @@
             ):
                 engine = OPENAI_API_ENGINE_GPT4_32K
             if OPENAI_API_TYPE is None or engine is None:
-                openai.api_key = OPENAI_API_KEY
-                openai.api_base = "https://api.openai.com/v1"
-                openai.api_version = None
-                openai.api_type = "open_ai"
-                response = openai.ChatCompletion.create(
-                    model=model,
-                    messages=messages,
-                    max_tokens=max_tokens,
-                    temperature=temperature,
-                    timeout=OPENAI_TIMEOUT,
-                    seed=SEED,
-                )
+                _call_openai(model, messages, max_tokens, temperature, OPENAI_API_KEY, "https://api.openai.com/v1", None, "open_ai")
                 return response["choices"][0].message.content
             # validity checks for MULTI_REGION_C

In [74]:
from rope.base.project import Project
from rope.contrib.autoimport import AutoImport

project = Project("../../sweepai")
autoimport = AutoImport(project)
# autoimport.generate_resource_cache()  # Generates a cache of the local modules, from the project you're working on
autoimport.generate_modules_cache(["../../sweepai", "."])  # Generates a cache of external modules
print(autoimport.import_assist("sweepai"))

[]
